In [7]:
# Import necessary packages
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchinfo
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

from matplotlib import pyplot as plt
import random

import os
from pathlib import Path
from typing import List

In [8]:
# Create a simple CNN model based on https://github.com/Sentdex/BCI/blob/master/training.py
class ProfessorXModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block_1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.Conv1d(in_channels=16, out_channels=16, kernel_size=2),
            torch.nn.ReLU(),
        )
        self.conv_block_2 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=16, out_channels=16, kernel_size=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
        )
        self.conv_block_3 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=16, out_channels=16, kernel_size=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=16*11, out_features=128),
            torch.nn.Linear(in_features=128, out_features=1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        snr = 1.
        std = torch.std(x)
        x += (2. * torch.rand(x.shape) - 1.) * 2. * std * snr

        result = self.classifier(self.conv_block_3(self.conv_block_2(self.conv_block_1(x))))
        return torch.squeeze(result)


In [9]:
PATH = 'Model/model_overall.pt'

model = torch.load(PATH)
#print(loaded_model['epoch'])
#print(loaded_model['model_state_dict'])
#print(loaded_model['optimizer_state_dict'])

In [10]:
#model = ProfessorXModel()
#model.load_state_dict(loaded_model['model_state_dict'])
#model.eval()

In [11]:
df = pd.read_csv('DATASET/CLEAN/TEST_DATASET.csv')
print(len(df))

119


In [12]:
id = 11
test = df.iloc[id, 1:].astype(float)

test = torch.from_numpy(test.values).unsqueeze(0).unsqueeze(0).to(torch.float32)

with torch.inference_mode():
    result = model(test)

if result.item() >= 0.7:
    command = 'RIGHT'
elif result.item() <= 0.3:
    command = 'LEFT'
else:
    command = 'UNDECIDED'

print("INPUT:", df.iloc[id, 0])
print("OUTPUT:",command)
print(result)

INPUT: LEFT
OUTPUT: LEFT
tensor(0.1016)
